In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np

In [2]:
# read in data
df = pd.read_csv('./Cluster0', delimiter=',')
print(df.head())

   Unnamed: 0  id    age  gender  height  ...  gluc  smoke  alco  active  cardio
0           1   1  20228       1     156  ...     1      0     0       1       1
1           3   3  17623       2     169  ...     1      0     0       1       1
2           6   9  22113       1     157  ...     1      0     0       1       0
3          22  32  23046       1     158  ...     2      0     0       1       1
4          24  35  16608       1     170  ...     1      0     0       1       1

[5 rows x 14 columns]


In [8]:
df_feat = df.iloc[:,1:-1] # eventhing but last column
y = df.loc[:, "cardio"] #last column
df_feat.drop("id", axis = 1)
df_feat.head() # age is in days

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,1,20228,1,156,85.0,140,90,3,1,0,0,1
1,3,17623,2,169,82.0,150,100,1,1,0,0,1
2,9,22113,1,157,93.0,130,80,3,1,0,0,1
3,32,23046,1,158,90.0,145,85,2,2,0,0,1
4,35,16608,1,170,68.0,150,90,3,1,0,0,1


In [9]:
from sklearn.preprocessing import StandardScaler
# standardize: mean = 0, std = 1
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_feat)
X = pd.DataFrame(scaled_features)
print(X)

             0         1         2   ...        9         10        11
0     -1.738593 -0.059287 -0.359705  ... -0.034555 -0.008634  0.491343
1     -1.738523 -1.240112  2.780053  ... -0.034555 -0.008634  0.491343
2     -1.738313  0.795168 -0.359705  ... -0.034555 -0.008634  0.491343
3     -1.737510  1.218089 -0.359705  ... -0.034555 -0.008634  0.491343
4     -1.737406 -1.700203 -0.359705  ... -0.034555 -0.008634  0.491343
...         ...       ...       ...  ...       ...       ...       ...
13411  1.751921 -0.710214 -0.359705  ... -0.034555 -0.008634  0.491343
13412  1.751956 -0.299078 -0.359705  ... -0.034555 -0.008634  0.491343
13413  1.751991  0.324198 -0.359705  ... -0.034555 -0.008634  0.491343
13414  1.752096  1.016374 -0.359705  ... -0.034555 -0.008634  0.491343
13415  1.752200  0.939315 -0.359705  ... -0.034555 -0.008634 -2.035240

[13416 rows x 12 columns]


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
#@title Default title text
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


for i in range(1,400):
    classifier = RandomForestClassifier(n_estimators=i, oob_score=False)
    classifier.fit(X_train, y_train)
    
    y_pred = classifier.predict(X_test)
    print("i: ", i, ";\tAccuracy:", accuracy_score(y_test, y_pred))
    print('Score: ', classifier.score(X_train, y_train))
#     print(classifier.oob_score_)
    print('---')


In [12]:
grid_params = {
    "n_estimators" : range(200, 300, 10)
}

In [13]:
RF = RandomForestClassifier(oob_score=False)
gs = GridSearchCV(RF, grid_params, verbose= 1, n_jobs= -1)
gs.fit(X_train, y_train)
gs.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


{'n_estimators': 280}

In [17]:
clusters = ["Cluster0", "Cluster1", "Cluster2", "Cluster3", "Cluster4", "Cluster5"]
RF = RandomForestClassifier(n_estimators = 280)
accuracyDict = {}
numEstimatorsDict = {}

for clust in clusters:
    # Preprocessing
    path_k = "./" + clust
    df = pd.read_csv(path_k, delimiter=',')
    df_feat = df.drop(["id", "cardio"], axis = 1)
    y = df.loc[:,"cardio"]
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df_feat)
    X = pd.DataFrame(scaled_features)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    
    # Train model
    gs = GridSearchCV(RF, grid_params, verbose= 1, n_jobs=-1)
    gs.fit(X_train, y_train)
    RF = RandomForestClassifier(n_estimators = gs.best_params_["n_estimators"])
    RF.fit(X_train, y_train)
    
    # Store accuracies
    y_pred = RF.predict(X_test)
    accuracyDict[clust] = accuracy_score(y_test, y_pred)
    
    # Store best # of estimtators
    numEstimatorsDict[clust] = gs.best_params_["n_estimators"]
    
print(accuracyDict)
print(numEstimatorsDict)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
{'Cluster0': 0.8084472049689441, 'Cluster1': 0.6638655462184874, 'Cluster2': 0.7596685082872928, 'Cluster3': 0.7522485690923958, 'Cluster4': 0.6818489289740699, 'Cluster5': 0.6925865733216652}
{'Cluster0': 250, 'Cluster1': 210, 'Cluster2': 260, 'Cluster3': 250, 'Cluster4': 220, 'Cluster5': 290}


In [18]:
print(accuracyDict)
print(numEstimatorsDict)

{'Cluster0': 0.8084472049689441, 'Cluster1': 0.6638655462184874, 'Cluster2': 0.7596685082872928, 'Cluster3': 0.7522485690923958, 'Cluster4': 0.6818489289740699, 'Cluster5': 0.6925865733216652}
{'Cluster0': 250, 'Cluster1': 210, 'Cluster2': 260, 'Cluster3': 250, 'Cluster4': 220, 'Cluster5': 290}


In [21]:
# Entire dataset

df = pd.read_csv("cardio_train.csv", delimiter = ",")
df.head()

,Unnamed: 0,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,label
0,0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0,2
1,1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1,0
2,2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1,3
3,3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1,0
4,4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0,3


In [22]:
targ = df['cardio']

In [24]:
df_feat = df.drop(["cardio", "id"], axis = 1)
df_feat.head()

,Unnamed: 0,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,label
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,2
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,0
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,3
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,0
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,3


In [25]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_feat)
X = pd.DataFrame(scaled_features)
print(X)

             0         1         2   ...        10        11        12
0     -1.732709 -0.434078  1.367567  ... -0.237475  0.494776 -0.191963
1     -1.732659  0.309349 -0.731226  ... -0.237475  0.494776 -1.612610
2     -1.732610 -0.246094 -0.731226  ... -0.237475 -2.021118  0.518361
3     -1.732560 -0.746033  1.367567  ... -0.237475  0.494776 -1.612610
4     -1.732511 -0.806399 -0.731226  ... -0.237475 -2.021118  0.518361
...         ...       ...       ...  ...       ...       ...       ...
68499  1.732257 -0.090927  1.367567  ... -0.237475  0.494776 -0.902286
68500  1.732307  1.270739 -0.731226  ... -0.237475  0.494776 -1.612610
68501  1.732356 -0.161420  1.367567  ...  4.210973 -2.021118  1.939008
68502  1.732406  1.201866 -0.731226  ... -0.237475 -2.021118 -1.612610
68503  1.732455  0.435752 -0.731226  ... -0.237475  0.494776  0.518361

[68504 rows x 13 columns]


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, targ)

In [27]:
grid_params

{'n_estimators': range(200, 300, 10)}

In [28]:
forest = RandomForestClassifier()
gs = GridSearchCV(forest, grid_params, verbose= 2)
gs.fit(X_train, y_train)
forest = RandomForestClassifier(n_estimators=gs.best_params_["n_estimators"])
forest.fit(X_train, y_train)
preds = forest.predict(X_test)
accuracy = accuracy_score(y_test, preds)
accuracy

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ...................................n_estimators=200; total time=  14.9s
[CV] END ...................................n_estimators=200; total time=  14.6s
[CV] END ...................................n_estimators=200; total time=  14.5s
[CV] END ...................................n_estimators=200; total time=  14.6s
[CV] END ...................................n_estimators=200; total time=  14.7s
[CV] END ...................................n_estimators=210; total time=  15.5s
[CV] END ...................................n_estimators=210; total time=  15.6s
[CV] END ...................................n_estimators=210; total time=  15.7s
[CV] END ...................................n_estimators=210; total time=  15.9s
[CV] END ...................................n_estimators=210; total time=  15.4s
[CV] END ...................................n_estimators=220; total time=  16.1s
[CV] END ...................................n_es

0.7225271516991708